In [1]:
import h5py
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import awkward as awk
import os
import argparse
from heputl import logging as heplog
import gc
import numpy as np

import src.reader as read
import src.generator as gene
import src.selection as sele
import src.string_constants as stco
import src.util as util

In [4]:
N = int(1e2)
feat_mc = ['jet_pt_lead','jet_eta_lead','jet_phi_lead', 'jet_GN2_pu_lead', 'jet_GN2_pb_lead', 'jet_GN2_pc_lead', 'jet_truthflavExtended_lead', 'wt', 'dsid']
dsids = stco.ds_ids_sig['tautau']        

In [11]:
print(f'list print {feat_mc}')

list print ['jet_pt_lead', 'jet_eta_lead', 'jet_phi_lead', 'jet_GN2_pu_lead', 'jet_GN2_pb_lead', 'jet_GN2_pc_lead', 'jet_truthflavExtended_lead', 'wt', 'dsid']


In [6]:
# read one chunk to get dtype
dsid = dsids[0]
samples = read.read_samples_for_dsid(dsid,N=N,filtered=True)
weights = util.compute_w_samples(samples, dsid)
samples['dsid'] = np.int32(dsid)
samples['wt'] = weights
samples = samples[feat_mc]

2024-06-10 16:27:55 [reader.py:read_samples_for_dsid] reading samples for dsid 512198 from /eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.512198.MGPy8EG_FxFx_Ztautau_3jets_HT2bias_BF.D_FTAG2.e8382_s3681_r13144_p5981.GN2v01_Nom_output.root


In [7]:
np.int32(dsids[0])

512198

In [8]:
dir(samples.type.content.contents)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [9]:
field_types = [s._primitive for s in samples.type.content.contents]

In [10]:
field_types

['float32',
 'float32',
 'float32',
 'float64',
 'float64',
 'float64',
 'int32',
 'float32',
 'int32']

In [135]:
import numpy as np
dtype = np.dtype(list(zip(samples.fields,field_types)))

In [136]:
outfile.close()

In [137]:
outfile = h5py.File('test.h5', 'w')

In [138]:
events = outfile.create_dataset("events", shape=(0,), maxshape=(None,), dtype=dtype)

In [139]:
arr = awk.to_numpy(samples) # 

In [140]:
n_batch = len(arr)

In [141]:
events.resize(len(events) + n_batch, axis=0)
events[-n_batch:] = arr

TypeError: Can't broadcast (100, 9) -> (100,)

In [142]:
arr

,jet_pt_lead,jet_eta_lead,jet_phi_lead,jet_GN2_pu_lead,jet_GN2_pb_lead,jet_GN2_pc_lead,jet_truthflavExtended_lead,wt,dsid
entry,,,,,,,,,
0,163825.390625,0.877953,-2.997266,0.764071,0.005337,0.161482,0,0.036745,506193
1,80900.328125,-0.518391,0.629305,0.114592,0.333799,0.547613,5,0.081536,506193
2,29493.052734,-0.923786,1.229845,0.002792,0.716793,0.276269,5,0.152672,506193
3,77220.539062,-1.026000,-2.609300,0.000046,0.999764,0.000190,5,0.050907,506193
4,35160.125000,-0.141383,2.725976,0.001161,0.997806,0.001031,5,-0.209198,506193
...,...,...,...,...,...,...,...,...,...
95,70783.046875,1.137471,-1.616167,0.000120,0.999799,0.000082,5,-0.106816,506193
96,87626.492188,-1.441725,1.401968,0.824275,0.019480,0.132676,0,0.039105,506193
97,59225.324219,0.445325,3.050378,0.000809,0.109908,0.885373,5,0.111674,506193


In [130]:
outfile.create_dataset("feature_names",data=[f.encode("ascii", "ignore") for f in feat_mc],dtype='S10')

<HDF5 dataset "feature_names": shape (9,), type "|S10">

In [26]:
samples.type.show()

100 * {
    jet_pt_lead: float32,
    jet_eta_lead: float32,
    jet_phi_lead: float32,
    jet_GN2_pu_lead: float64,
    jet_GN2_pb_lead: float64,
    jet_GN2_pc_lead: float64,
    jet_truthflavExtended_lead: int32,
    wt: float32,
    dsid: string
}


In [27]:
dir(samples.type)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_behavior',
 '_content',
 '_length',
 '_str',
 'behavior',
 'content',
 'is_equal_to',
 'length',
 'show']

In [29]:
samples.type.behavior

In [30]:
samples.type.content

RecordType([NumpyType('float32'), NumpyType('float32'), NumpyType('float32'), NumpyType('float64'), NumpyType('float64'), NumpyType('float64'), NumpyType('int32'), NumpyType('float32'), ListType(NumpyType('uint8', parameters={'__array__': 'char'}), parameters={'__array__': 'string'})], ['jet_pt_lead', 'jet_eta_lead', 'jet_phi_lead', 'jet_GN2_pu_lead', 'jet_GN2_pb_lead', 'jet_GN2_pc_lead', 'jet_truthflavExtended_lead', 'wt', 'dsid'])

In [31]:
samples.type

ArrayType(RecordType([NumpyType('float32'), NumpyType('float32'), NumpyType('float32'), NumpyType('float64'), NumpyType('float64'), NumpyType('float64'), NumpyType('int32'), NumpyType('float32'), ListType(NumpyType('uint8', parameters={'__array__': 'char'}), parameters={'__array__': 'string'})], ['jet_pt_lead', 'jet_eta_lead', 'jet_phi_lead', 'jet_GN2_pu_lead', 'jet_GN2_pb_lead', 'jet_GN2_pc_lead', 'jet_truthflavExtended_lead', 'wt', 'dsid']), 100, None)

In [39]:
dir(samples.type.content.contents[0])

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_typestr',
 '_is_equal_to',
 '_parameters',
 '_primitive',
 '_repr_args',
 '_str',
 '_str_categorical_begin',
 '_str_categorical_end',
 '_str_parameters',
 '_str_parameters_exclude',
 'copy',
 'is_equal_to',
 'parameter',
 'parameters',
 'primitive',
 'show']

In [38]:
print(samples.type.content.contents)

[NumpyType('float32'), NumpyType('float32'), NumpyType('float32'), NumpyType('float64'), NumpyType('float64'), NumpyType('float64'), NumpyType('int32'), NumpyType('float32'), ListType(NumpyType('uint8', parameters={'__array__': 'char'}), parameters={'__array__': 'string'})]


In [65]:
outfile = h5py.File('test.h5', 'w')

In [66]:
samples.fields

['jet_pt_lead',
 'jet_eta_lead',
 'jet_phi_lead',
 'jet_GN2_pu_lead',
 'jet_GN2_pb_lead',
 'jet_GN2_pc_lead',
 'jet_truthflavExtended_lead',
 'wt',
 'dsid']

In [68]:
list(zip(samples.fields,dtypes))

[('jet_pt_lead', 'float32'),
 ('jet_eta_lead', 'float32'),
 ('jet_phi_lead', 'float32'),
 ('jet_GN2_pu_lead', 'float64'),
 ('jet_GN2_pb_lead', 'float64'),
 ('jet_GN2_pc_lead', 'float64'),
 ('jet_truthflavExtended_lead', 'int32'),
 ('wt', 'float32'),
 ('dsid', 'str')]